In [2]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pymongo
import requests
import os
import pandas as pd
from selenium import webdriver
import time

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.nhl_db
collection = db.articles

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
# Retrieve the parent divs for all articles
results = soup.find('div', class_='features')

news_title = results.find('div', class_='content_title').text
print(news_title)

news_body = results.find('div', class_='rollover_description').text
print(news_body)



Opportunity Hunkers Down During Dust Storm




It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home. 







In [7]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [9]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

top_thing = soup.find('div', class_='default floating_text_area ms-layer')
# print(top_thing)

footer_thing = top_thing.find('footer')
# print(footer_thing)

featured_image_url = 'https://www.jpl.nasa.gov'+ footer_thing.find('a')['data-fancybox-href']
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16028_ip.jpg


In [10]:
twitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter)

twitter_html = browser.html
twitter_soup = BeautifulSoup(twitter_html, 'html.parser')

tweet_text = twitter_soup.find('div', 'js-tweet-text-container').text
print(tweet_text)


Sol 2227 (2018-11-11), high -3C/26F, low -72C/-97F, pressure at 8.63 hPa, daylight 06:21-18:38



In [11]:
mars_facts = 'https://space-facts.com/mars/'
browser.visit(mars_facts)    

mars_table = pd.read_html(mars_facts)
mars_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [13]:
mars_df = mars_table[0]
mars_df.columns = ['Description', 'Values']
mars_df.head()

,Description,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [22]:
# Reset index 
mars_df.set_index('Description')
mars_df.head()

,Description,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [25]:
# convert dataframe to html
mars_facts = mars_df.to_html()
mars_facts.replace("\n", "")
mars_df.to_html('mars_facts.html')

In [26]:
valles_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"
cerberus_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
schiaparelli_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
syrtis_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"

browser.visit(cerberus_url)
cerberus_html = browser.html
cerberus_soup = BeautifulSoup(cerberus_html, 'lxml')

browser.visit(valles_url)
valles_html = browser.html
valles_soup = BeautifulSoup(valles_html, 'lxml')

browser.visit(schiaparelli_url)
schiaparelli_html = browser.html
schiaparelli_soup = BeautifulSoup(schiaparelli_html, 'lxml')

browser.visit(syrtis_url)
syrtis_html = browser.html
syrtis_soup = BeautifulSoup(syrtis_html, 'lxml')

cerberus_image = cerberus_soup.find('div', class_='downloads').find('a')['href']
print(cerberus_image)

valles_image = valles_soup.find('div', class_='downloads').find('a')['href']
print(valles_image)

schiaparelli_image = schiaparelli_soup.find('div', class_='downloads').find('a')['href']
print(schiaparelli_image)

syrtis_image = syrtis_soup.find('div', class_='downloads').find('a')['href']
print(syrtis_image)

hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": valles_image},
    {"title": "Cerberus Hemisphere", "img_url": cerberus_image},
    {"title": "Schiaparelli Hemisphere", "img_url": schiaparelli_image},
    {"title": "Syrtis Major Hemisphere", "img_url": syrtis_image}
]

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
